# Pre-semester Assignment 
- Author: 楊晴雯
- IKM Lab 111-1 pre-semester assignment
- Date Created: Aug 27, 2022
- Task: 
實作一個文字分類器，dataset使用AGNews，並且比較用pre-trained word embeddings在表現上的差異
  - Model Architecture: `Keras`
  - Data Preprocessing Pipeline: `torchtext` 
  - Pretrained Embedding: `FastText`
- Dataset EDA
- Eval metrics: F1, accuracy, confusion_matrix
- Note:
```
reproducibility (seeding)
track report (wandb)


```
- References
```
https://github.com/aayushs879/AG-News-Sentence-Classification/blob/master/Sentence_Classification.ipynb
https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-simple-guide-to-text-classification
https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/
https://colab.research.google.com/drive/1kvX7XTJEVLmRYT9w37OjWsIl1gCRZL_U?authuser=2#scrollTo=f3ad0878
https://www.kaggle.com/code/ishandutta/ag-news-classification-lstm
https://ai.stackexchange.com/questions/9756/lstm-language-model-not-working
https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset/code
https://www.google.com/search?q=ag+news+lstm+&biw=1440&bih=789&sxsrf=ALiCzsbkQlp_OPsqUuq3qdEpNLHfeDNz7Q%3A1661960780835&ei=TIIPY9XPMtbf2roPvt6eyAg&ved=0ahUKEwjVpeOvtvH5AhXWr1YBHT6vB4kQ4dUDCA4&uact=5&oq=ag+news+lstm+&gs_lcp=Cgdnd3Mtd2l6EAMyBAgjECc6BwgAEEcQsAM6BQgAEIAEOgYIABAeEBY6CwgAEB4QDxDJAxAWOgoIABCABBCHAhAUOggIABAeEBYQCjoICAAQHhAIEA1KBAhBGABKBAhGGABQ0wNY3A5gqRBoAXABeACAAU2IAesDkgEBN5gBAKABAcgBCMABAQ&sclient=gws-wiz
```


In [1]:
!nvidia-smi

Thu Sep  1 11:48:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    13W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ROOT = '/content/drive/MyDrive/111-1成大課程/ML/'
!pip -q install datasets
!pip -q install torch 
!pip -q install Dataset
!pip -q install tqdm 

     |████████████████████████████████| 365 kB 7.7 MB/s 
     |████████████████████████████████| 120 kB 55.1 MB/s 
     |████████████████████████████████| 115 kB 62.8 MB/s 
     |████████████████████████████████| 212 kB 55.3 MB/s 
     |████████████████████████████████| 127 kB 62.7 MB/s 
     |████████████████████████████████| 209 kB 7.3 MB/s 
     |████████████████████████████████| 78 kB 8.0 MB/s 


### Downloading Dataset 
- AG News (AG’s News Corpus) is a subdataset of AG's corpus of news articles constructed by assembling titles and description fields of articles from the 4 largest classes (“World”, “Sports”, “Business”, “Sci/Tech”) of AG’s Corpus. The AG News contains 30,000 training and 1,900 test samples per class.
- Label Mapping: ```“World”: 0,“Sports”: 1,“Business”: 2,“Sci/Tech”: 3```
- Even (Equal) class distribution 
- No need to do train test split 


In [4]:
# average sequence length, token length 
from collections import defaultdict
import numpy as np 
from datasets import list_datasets, load_dataset
agnews_dataset = load_dataset('ag_news')
trainset, testset = agnews_dataset['train'], agnews_dataset['test']

ClassSeqLen = defaultdict(list)
LabelMapping = {0:"World",
                1:"Sports",
                2:"Business",
                3:"Sci/Tech"}

def get_stats(split):
    for i in range(len(split)):
        data = split[i]
        text, label = data['text'], data['label']
        ClassSeqLen[label].append(len(text))
    for k, v in ClassSeqLen.items():
        avg, var = sum(v)/len(v), np.var(v)
        avg, var = round(avg, 2), round(var, 2)
        print(f'  {LabelMapping[k]}: Average: {avg}, Variance: {var}')
print('== Sequence Length Stats by Class: == ')
print('-- train -- ') 
get_stats(trainset)
print('-- test -- ') 
get_stats(testset)

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

== Sequence Length Stats by Class: == 
-- train -- 
  Business: Average: 241.41, Variance: 4129.48
  Sci/Tech: Average: 237.22, Variance: 6763.04
  Sports: Average: 224.65, Variance: 2550.1
  World: Average: 242.63, Variance: 4048.9
-- test -- 
  Business: Average: 241.33, Variance: 4103.19
  Sci/Tech: Average: 237.23, Variance: 6769.03
  Sports: Average: 224.58, Variance: 2541.67
  World: Average: 242.49, Variance: 4040.47


## Configuration

In [5]:
Config = {
    'BATCHSIZE': 200,
    'LR':1e-4,
    'MAXLEN':50,
    'DATASET':'AGNews', 
    'EPOCHS':10,
    'WEIGHT_DECAY':1e-5,  #default:0
    'DROPOUT':0.3, 
    'CLIP_GRAD':5.0,
    'NUMCHOICE':4,
    'LSTM_HDIM':256, 
    'EMBDIM':32,
    'HIDDIM':32
}

## Preprocessing & Tokenization

In [6]:
import pandas as pd
import numpy as np 
import random
import torch 
import os
import torch.nn as nn
import re
import joblib
from tensorflow import keras
import torchtext

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
def clean(x):
  text,label = x['text'], x['label']
  return {'text': re.sub(r'[~`\-!@#$%^&*():;"{}_/?><\|.,`0-9\\]',' ',text), # regex replace multiple spaces to one 
          'label':label}
trainset = trainset.map(clean)
testset = testset.map(clean)

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

In [9]:
# https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-simple-guide-to-text-classification
# https://pytorch.org/text/stable/vocab.html#build-vocab-from-iterator
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from nltk.corpus import stopwords

tokenizer = get_tokenizer("basic_english")
stopwords = set(stopwords.words('english'))
def build_vocab(datasets):
      for dataset in [trainset, testset]:
        for data in dataset:
            text = data['text']
            yield [x.lower() for x in tokenizer(text) if x.lower() not in stopwords]
vocabs = build_vocab_from_iterator(build_vocab([trainset, testset]), 
                                   specials=["[UNK]", "[PAD]"], max_tokens = None)
vocabs.set_default_index(vocabs["[UNK]"]) # if not in vocab, set it as [UNK]

In [10]:
print(f'[UNK]: {vocabs["[UNK]"]}')
print(f'[PAD]: {vocabs["[PAD]"]}')
print(f'Vocab Size: {len(vocabs.get_itos())}')

[UNK]: 0
[PAD]: 1
Vocab Size: 63613


### Frequency-based Vectorizer

- This approach loses text sequence information as it encodes the sequence into a (,vocab_size) sequence. 
- `CountVectorzier, TfidfVectorizer`

### Dense Representation
- simply map word into index 
- train by embeddings

In [11]:
# trainy = np.array([x['label'] for x in trainset], dtype=int)
# testy = np.array([x['label'] for x in testset], dtype=int)

In [12]:
# [UNK] and [PAD]
Config['VOCAB_SIZE'] = len(vocabs)

In [13]:
import re
def tokenize(text):
    maxlen = Config['MAXLEN']
    text = text.split(' ')
    text = text[:maxlen]                # truncation
    tokenized_x = np.ones(maxlen)       # padding: np.ones since [PAD]: 1 
    tokenized_x[:len(text)] = np.array([vocabs[v] for v in text])
    return tokenized_x
padded_trainX = np.vstack([tokenize(x['text']) for x in trainset])
padded_trainX  = padded_trainX.astype(int)
trainy = np.array([x['label'] for x in trainset], dtype = int) 

padded_testX = np.vstack([tokenize(x['text']) for x in testset])
padded_trainX  = padded_trainX.astype(int)
testy = np.array([x['label'] for x in testset], dtype = int) 

In [14]:
# Verifying shape
print(padded_trainX.shape) # texts: (data_size, maxlen)
print(trainy.shape) # labels: (data_size, )
print(padded_testX.shape)
testy.shape 

(120000, 50)
(120000,)
(7600, 50)


(7600,)

In [15]:
# Verify Tokenization
split = trainset
r = random.randint(0, len(split))
detokenizer = vocabs.get_itos()
print(len(vocabs.get_itos()))
# “World”: 0,“Sports”: 1,“Business”: 2,“Sci/Tech”: 3
print(trainy[r])
print(trainset[r])
print(padded_trainX[r,:])
print([detokenizer[x] for x in padded_trainX[r,:]])

63613
1
{'text': 'Blue Jays to buy SkyDome for         million The Toronto Blue Jays agreed to buy SkyDome  the team     s ballpark  for about         million  The        seat stadium  which opened in       cost about       million to construct and was mostly funded by taxpayers ', 'label': 1}
[   0    0    0  200    0    0    0    0    0    0    0    0    0    0
   33    0    0    0    0  224    0  200    0    0    0   61    0    0
    0    0    0 6760    0    0    0    0    0    0    0    0    0    0
    0   33    0    0    0    0    0    0]
['[UNK]', '[UNK]', '[UNK]', 'buy', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'million', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'agreed', '[UNK]', 'buy', '[UNK]', '[UNK]', '[UNK]', 'team', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'ballpark', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'million', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK

## [w/o pretrained] Model Architecture & Training

In [16]:
# define network 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout

model = Sequential()
model.add(Embedding(input_dim= Config['VOCAB_SIZE'], 
                    output_dim= Config['EMBDIM'],  
                    input_length=Config['MAXLEN'], name = 'embedding'))
model.add(LSTM(Config['LSTM_HDIM'],name = 'lstm')) 
model.add(Dense(Config['HIDDIM'], name = 'fc')) # 這裡不應該有softmax
model.add(Dropout(Config['DROPOUT'], name = 'dropout'))
model.add(Dense(Config['NUMCHOICE'], activation="softmax", name = 'classifier'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 32)            2035616   
                                                                 
 lstm (LSTM)                 (None, 256)               295936    
                                                                 
 fc (Dense)                  (None, 32)                8224      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 classifier (Dense)          (None, 4)                 132       
                                                                 
Total params: 2,339,908
Trainable params: 2,339,908
Non-trainable params: 0
_________________________________________________________________


In [17]:
from sklearn.metrics import f1_score
optimizer = keras.optimizers.Adam(lr=Config['LR'])
model.compile(optimizer, 
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])
history = model.fit(padded_trainX, trainy, 
          batch_size = Config['BATCHSIZE'],
          epochs = Config['EPOCHS'], shuffle = True, validation_data=(padded_testX, testy))

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


600/600 [==============================] - 14s 14ms/step - loss: 1.0764 - accuracy: 0.4907 - val_loss: 0.8738 - val_accuracy: 0.6283
Epoch 2/10
600/600 [==============================] - 7s 12ms/step - loss: 0.7738 - accuracy: 0.6870 - val_loss: 0.7321 - val_accuracy: 0.7196
Epoch 3/10
600/600 [==============================] - 7s 12ms/step - loss: 0.6070 - accuracy: 0.7823 - val_loss: 0.6070 - val_accuracy: 0.7808
Epoch 4/10
600/600 [==============================] - 7s 12ms/step - loss: 0.5152 - accuracy: 0.8241 - val_loss: 0.5581 - val_accuracy: 0.7979
Epoch 5/10
600/600 [==============================] - 7s 12ms/step - loss: 0.4568 - accuracy: 0.8471 - val_loss: 0.5438 - val_accuracy: 0.8121
Epoch 6/10
600/600 [==============================] - 7s 12ms/step - loss: 0.4181 - accuracy: 0.8635 - val_loss: 0.5342 - val_accuracy: 0.8184
Epoch 7/10
600/600 [==============================] - 7s 12ms/step - loss: 0.3837 - accuracy: 0.8768 - val_loss: 0.5039 - val_accuracy: 0.8308
Epoch 8/1

In [18]:
history

## (w/ pretrained) Model + Training

In [19]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [20]:
ROOT = '/content/drive/MyDrive/111-1成大課程/ML/'
FastTextModelPath = f'{ROOT}/agnews_fastext.model'
from gensim.models import fasttext
ftmodel = fasttext.FastText.load(FastTextModelPath)

In [21]:
mean = 0 
std = 1 
# [PAD] keras 需要設定PAD的index嗎？
pretrained_embeddings = np.random.normal(loc = mean, scale = std, 
                                    size = (Config['VOCAB_SIZE'], Config['EMBDIM']))
for idx, word in enumerate(vocabs.get_itos()):
    embedding_vector = ftmodel.wv[word]
    if embedding_vector is not None:
          pretrained_embeddings[idx] = embedding_vector
    # word not found will be initialized follow random normal distribution 

In [22]:
ftmodel.wv.most_similar('computer')

[('megacomputer', 0.9801978468894958),
 ('cmputer', 0.9546562433242798),
 ('computersand', 0.9464218020439148),
 ('compute', 0.9461243152618408),
 ('minicomputer', 0.9381483793258667),
 ('compu', 0.9298766851425171),
 ('computershare', 0.9191988706588745),
 ('computers', 0.9171096086502075),
 ('compusa', 0.9065960049629211),
 ('compulsory', 0.8953425884246826)]

In [23]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout

model = Sequential()
model.add(Embedding(input_dim= Config['VOCAB_SIZE'], 
                    weights=[pretrained_embeddings],
                    trainable = True,
                    output_dim= Config['EMBDIM'],  
                    input_length=Config['MAXLEN'], name = 'embedding'))
model.add(LSTM(Config['LSTM_HDIM'],name = 'lstm')) 
model.add(Dense(Config['HIDDIM'], name = 'fc')) # 這裡不應該有softmax
model.add(Dropout(0.3, name = 'dropout'))
model.add(Dense(Config['NUMCHOICE'], activation="softmax", name = 'classifier'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 32)            2035616   
                                                                 
 lstm (LSTM)                 (None, 256)               295936    
                                                                 
 fc (Dense)                  (None, 32)                8224      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 classifier (Dense)          (None, 4)                 132       
                                                                 
Total params: 2,339,908
Trainable params: 2,339,908
Non-trainable params: 0
_________________________________________________________________


In [24]:
from sklearn.metrics import f1_score
optimizer = keras.optimizers.Adam(learning_rate=Config['LR'])
model.compile(optimizer, 
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])
history = model.fit(padded_trainX, trainy, 
          batch_size = Config['BATCHSIZE'],
          epochs = Config['EPOCHS'], shuffle = True, validation_data=(padded_testX, testy))

Epoch 1/10
600/600 [==============================] - 12s 16ms/step - loss: 0.6221 - accuracy: 0.7782 - val_loss: 0.5127 - val_accuracy: 0.8174
Epoch 2/10
600/600 [==============================] - 9s 15ms/step - loss: 0.5014 - accuracy: 0.8248 - val_loss: 0.4901 - val_accuracy: 0.8251
Epoch 3/10
600/600 [==============================] - 7s 12ms/step - loss: 0.4802 - accuracy: 0.8316 - val_loss: 0.4806 - val_accuracy: 0.8250
Epoch 4/10
600/600 [==============================] - 8s 13ms/step - loss: 0.4660 - accuracy: 0.8378 - val_loss: 0.4662 - val_accuracy: 0.8289
Epoch 5/10
600/600 [==============================] - 8s 13ms/step - loss: 0.4521 - accuracy: 0.8429 - val_loss: 0.4616 - val_accuracy: 0.8328
Epoch 6/10
600/600 [==============================] - 8s 13ms/step - loss: 0.4419 - accuracy: 0.8462 - val_loss: 0.4464 - val_accuracy: 0.8361
Epoch 7/10
600/600 [==============================] - 7s 12ms/step - loss: 0.4334 - accuracy: 0.8497 - val_loss: 0.4422 - val_accuracy: 0.840